# Case study of metaprogramming: interval constraint propagation 

Perhaps the best way to learn metaprogramming is through an example. 
Let's look at **interval constraint propagation**, as implemented in the author's
[`IntervalConstraintProgramming.jl`](https://github.com/dpsanders/IntervalConstraintProgramming.jl) package.

This is less complicated than it sounds. The idea is to find a way of taking a declarative **constraint**, like `x^2 + y^2 <= 1`, representing a set `S`, and **turning it into actual code** that operates on interval variables `x` and `y` representing a box `B` in the plane, and *contracts* it (squashes it down) to a smaller subbox `B'` that still contains the part of `B` that is inside `S`.

The method takes the AST (Abstract Syntax Tree) of the expression, and unfolds it in two directions: first "forward", from the leaves of the tree towards the root, and then backwards, incorporating the constraints. We will present a simplified version of the code at https://github.com/dpsanders/IntervalConstraintProgramming.jl/blob/master/src/contractor.jl

Let's take the example of the expression

In [16]:
ex = :(x^2 + y^2)

:(x ^ 2 + y ^ 2)

We would like to introduce new variables at each node in the tree, from the "bottom" (i.e. the leaves) up, giving the result

    z1 = x^2
    z2 = y^2
    z3 = z1 + z2
    
These variables will be used later in the backwards pass to pass back information about the constraint that `z3` must be in the interval `[0,1]`.

When we define an expression, Julia parses it into its AST:

In [60]:
Meta.show_sexpr(ex)

(:call, :+, (:(=), :z16, (:call, :^, :x, 2)), (:(=), :z17, (:call, :^, :y, 2)))

## Forward pass

In [61]:
workspace()

We first write a function to generate new unique symbols. (We could use `gensym()` from `Base`, but the result is unreadable for humans):

In [142]:
const _variable_number_ = [0]

makesymbol() = symbol("_z_", _variable_number_[1]+=1)

makesymbol (generic function with 1 method)

In [144]:
makesymbol()

:_z_2

We will make a recursive function that takes a sub-expression and returns two things: (i) the new variable at the top of that subexpression; and (ii) the code that has been generated so far.

In [147]:
unfold(ex) = (ex, quote end)  # catch-all for symbols and constants


function unfold(ex::Expr)

    ex.head != :call && throw(ArgumentError("Unknown expression type: $ex"))
    
    new_code = quote end   # empty expression

    op = ex.args[1]
    
    # do child nodes *first*:
    var1, code1 = traverse(ex.args[2])
    var2, code2 = traverse(ex.args[3])
        
    new_var = makesymbol()
    new_line = :($new_var = $op($var1, $var2))   
    
    append!(new_code.args, code1.args)
    append!(new_code.args, code2.args)
    push!(new_code.args, new_line)
    
    return new_var, new_code

end


unfold (generic function with 2 methods)

## Backward pass

The top-level variable must now be constrained:

In [148]:
ex = :(x^2 + y^2)
var, code = unfold(ex)

(:_z_5,quote 
    _z_3 = x ^ 2
    _z_4 = y ^ 2
    _z_5 = _z_3 + _z_4
end)

In [154]:
interval = :(Interval(0,1))

constraint_code = :($var = $var ∩ $interval)
push!(code.args, constraint_code)

4-element Array{Any,1}:
 :(_z_3 = x ^ 2)               
 :(_z_4 = y ^ 2)               
 :(_z_5 = _z_3 + _z_4)         
 :(_z_5 = _z_5 ∩ Interval(0,1))

Now we wish to go backwards through the generated code, and use reverse-mode functions to propagate the constraints.
For example, if `z5 = z3 + z4`, then `z3 = z5 - z4`, so we can *constrain* `z3` accordingly by

    z3 = z3 ∩ (z5 - z4)
    
Here, ∩ is an operation defined on intervals in the [`ValidatedNumerics.jl package`](https://github.com/dpsanders/ValidatedNumerics.jl)

We will assume that there are so-called "reverse mode" functions defined that do these operations; see https://github.com/dpsanders/IntervalConstraintProgramming.jl/blob/master/src/reverse_mode.jl

We will then replace `z5 = z3 + z4` by `(z5,z3,z4) = reverse_add(z5, z3, z4)` with the following function

To find out how to do this, we do

In [167]:
ex = :(z = x + y)
dump(ex)

Expr 
  head: Symbol =
  args: Array(Any,(2,))
    1: Symbol z
    2: Expr 
      head: Symbol call
      args: Array(Any,(3,))
        1: Symbol +
        2: Symbol x
        3: Symbol y
      typ: Any
  typ: Any


This tells us which elements to use:

In [168]:
var1 = ex.args[1]
op = ex.args[2].args[1]
var2, var3 = ex.args[2].args[2:3]

2-element Array{Any,1}:
 :x
 :y

In [156]:
const reverse_ops = Dict(:+ => :reverse_add,
                            :- => :reverse_sub)

Dict{Symbol,Symbol} with 2 entries:
  :+ => :reverse_add
  :- => :reverse_sub

In [163]:
function make_reverse(ex)
    # a = b op c
    
    var1 = ex.args[1]
    op = ex.args[2].args[1]
    var2, var3 = ex.args[2].args[2:3]
    
    reverse_op = reverse_ops[op]
    
    return :( ($var1, $var2, $var3) = $reverse_op($var1, $var2, $var3) )
    
end

make_reverse (generic function with 1 method)

In [165]:
make_reverse(ex)

:((z,x,y) = reverse_add(z,x,y))

## MacroTools.jl 


The extraction of the variables was pretty ugly. There is a solution to this: the `MacroTools.jl` package.

In [169]:
using MacroTools

In [170]:
ex

:(z = x + y)

In [175]:
(op, var1, var2, var3) = @match ex begin
    (var1_ = op_(var2_, var3_)) => (op, var1, var2, var3)
end

(:+,:z,:x,:y)

**Exercise**: Finish writing the backwards pass.